In [ ]:
import os
import pandas as pd
import dtale
import sys, os
project_root = os.path.abspath("..")  # adjust based on your directory structure
if project_root not in sys.path:
    sys.path.append(project_root)
    
from FeatureCleaning.CleanDSDTale import export_clean_df


def main():
    
    # 1. Load the cleaned issue-level DataFrame
    df = export_clean_df()
    
    # 2. Compute time-to-resolution in hours
    df['time_to_resolution'] = (
        pd.to_datetime(df['fields.resolutiondate']) - 
        pd.to_datetime(df['fields.created'])
    ).dt.total_seconds() / 3600  # in hours

    item_counts_per_project = df.groupby("fields.project.id").size().reset_index(name="item_count")

    # 3. Aggregate overall project-level features (transformations and mean/median values)
    agg_df = df.groupby(['fields.project.id', 'fields.project.name']).agg({
        'time_to_resolution': ['mean', 'median', 'min', 'max', 'sum'],
        'changelog_count_status': ['sum', 'mean'],
        'changelog_count_assignee': ['sum', 'mean'],
        'comment_count': ['sum', 'mean'],
    }).reset_index()
    
    # Flatten the MultiIndex columns
    agg_df.columns = [
        '_'.join(col).strip() if col[1] else col[0]
        for col in agg_df.columns.values
    ]

    df = pd.merge(agg_df, item_counts_per_project, on="fields.project.id", how="left")
    
    # 6. Optionally display the final DataFrame using D-Tale for interactive exploration
    print(df.head())
    d = dtale.show(df, ignore_duplicate=True, allow_cell_edits=False)
    d.open_browser()

if __name__ == "__main__":
    main()


Processing repository: MongoDB ...
Found 137172 total issues in 'MongoDB'. Processing in batches of 500...
Using fixed maximum of 2000 records. Will retrieve 2000 issues.
